# 建模调参
## 4.1 学习目标

- 学习在金融分控领域常用的机器学习模型
- 学习机器学习模型的建模过程与调参流程
- 完成相应学习打卡任务 



## 4.2 内容介绍

- 逻辑回归模型：

  - 理解逻辑回归模型；
  - 逻辑回归模型的应用；
  - 逻辑回归的优缺点；

- 树模型：

  - 理解树模型；
  - 树模型的应用；
  - 树模型的优缺点；

- 集成模型

  - 基于bagging思想的集成模型
    - 随机森林模型
  - 基于boosting思想的集成模型
    - XGBoost模型
    - LightGBM模型
    - CatBoost模型

- 模型对比与性能评估：

  - 回归模型/树模型/集成模型；
  - 模型评估方法；
  - 模型评价结果；

- 模型调参：

  - 贪心调参方法；

  - 网格调参方法；

  - 贝叶斯调参方法

In [1]:
import pandas as pd
import numpy as np
import warnings
import os
import seaborn as sns
import matplotlib.pyplot as plt

Illegal line #1
	"{\rtf1\ansi\ansicpg936\cocoartf1671\cocoasubrtf200
"
	in file "/Users/ousei/.matplotlib/matplotlibrc"
Illegal line #2
	"{\fonttbl\f0\fnil\fcharset0 Menlo-Regular;}
"
	in file "/Users/ousei/.matplotlib/matplotlibrc"
Illegal line #3
	"{\colortbl;\red255\green255\blue255;\red0\green0\blue0;\red255\green255\blue255;}
"
	in file "/Users/ousei/.matplotlib/matplotlibrc"
Illegal line #4
	"{\*\expandedcolortbl;;\cssrgb\c0\c0\c0;\cssrgb\c100000\c100000\c100000;}
"
	in file "/Users/ousei/.matplotlib/matplotlibrc"
Illegal line #5
	"\paperw11900\paperh16840\margl1440\margr1440\vieww10800\viewh8400\viewkind0
"
	in file "/Users/ousei/.matplotlib/matplotlibrc"
Illegal line #6
	"\deftab720
"
	in file "/Users/ousei/.matplotlib/matplotlibrc"
Illegal line #7
	"\pard\pardeftab720\sl260\partightenfactor0
"
	in file "/Users/ousei/.matplotlib/matplotlibrc"
Illegal line #9
	"\f0\fs22 \cf2 \cb3 \expnd0\expndtw0\kerning0
"
	in file "/Users/ousei/.matplotlib/matplotlibrc"

Bad key "\outl0\stroke

In [2]:
"""
sns 相关设置
@return:
"""
# 声明使用 Seaborn 样式
sns.set()
# 有五种seaborn的绘图风格，它们分别是：darkgrid, whitegrid, dark, white, ticks。默认的主题是darkgrid。
sns.set_style("whitegrid")
# 有四个预置的环境，按大小从小到大排列分别为：paper, notebook, talk, poster。其中，notebook是默认的。
sns.set_context('talk')
# 中文字体设置-黑体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False
# 解决Seaborn中文显示问题并调整字体大小
sns.set(font='SimHei')

In [ ]:
#reduce_mem_usage函数通过调整数据类型,帮助减少数据在内存中占用的空间
def reduce_mem_usage(df):
    start_men = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f}MB'.format(start_men))

    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
data = pd.read_csv('dataset/data_for_model.csv')
data = reduce_mem_usage(data)